In [1]:
# 테스트 세트를 사용하지 않으면 모델이 과대적합인지, 과소적합인지 알기 어렵습니다.
# 이 떄, 훈련세트를 또 나누어서 검증세트를 만듭니다.
# 지금까지 훈련세트 80%, 테스트 세트 20%를 만들었다면, 이제부터는 훈련세트 60%, 검증세트 20%, 테스트세트 20%를 만들겁니다.
# 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가합니다. 이런 식으로 테스트하고 싶은 매개변 수를 바꿔가며 가장 좋은 모델을 고릅니다. 
# 그다음 이 매개변수를 사용해 훈련 세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련합니다. 그리고 마지막에 테스트 세트에서 최종 점수를 평가합니다. 
# 아마도 실전에 투입했을 때 테스트 세트의 점수와 비슷한 성능을 기대할 수 있을 것입니다.
# 그럼 이전 절에 사용했던 데이터를 다시 불러와서 검증 세트를 만들어 보겠습니다. 먼저 판다스로 CSV 데이터를 읽습니다.

In [3]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [4]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,
                                                                      target,
                                                                      random_state=42,
                                                                      test_size=0.2)

In [6]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input,
                                                                train_target,
                                                                test_size=0.2,
                                                                random_state=42)

In [7]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [8]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=42)
dtc.fit(sub_input, sub_target)
print(dtc.score(sub_input, sub_target))
print(dtc.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [10]:
# 보통 많은 데이터를 훈련에 사용할 수록 좋은 모델이 만들어집니다. 하지만 검증 세트를 너무 조금 떼어 놓으면 검증 점수가
# 들쭉날쭉하고 불안정합니다. 이럴 때, 교차 검증을 이용하면 안정적인 검증 점수를 얻고 훈련에 많은 데이터를 사용할 수 있습니다.
from sklearn.model_selection import cross_validate
scores = cross_validate(dtc, train_input, train_target)
print(scores)

{'fit_time': array([0.00936913, 0.00975585, 0.00950694, 0.00942421, 0.00873017]), 'score_time': array([0.00142598, 0.00097108, 0.00107312, 0.00085592, 0.00081587]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [12]:
# 회귀 : KFold, 분류 : StratifiedKFold
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dtc, train_input, train_target, cv=StratifiedKFold())
print(scores)

{'fit_time': array([0.01048708, 0.00875282, 0.00780487, 0.0084691 , 0.00790811]), 'score_time': array([0.00164294, 0.00108504, 0.00065422, 0.00071073, 0.00070405]), 'test_score': array([0.87019231, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [16]:
import numpy as np
scores = cross_validate(dtc, train_input, train_target, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42))
print(np.mean(scores['test_score']))
print(scores['test_score'])

0.8581873425226026
[0.83461538 0.88461538 0.85384615 0.85384615 0.84615385 0.87307692
 0.86153846 0.85549133 0.85163776 0.86705202]


In [17]:
from sklearn.model_selection import GridSearchCV
params = {
    'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid=params, n_jobs=-1)

In [18]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [21]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [23]:
gs.cv_results_['mean_test_score']

array([0.86800067, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [25]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [31]:
params = {
    'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
    'max_depth':np.arange(5, 20, 1),
    'min_samples_split':np.arange(2, 100, 10)
}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': np.int64(14), 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': np.int64(12)}


/Users/hack/가상 환경/statistic/lib/python3.9/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [33]:
gs.cv_results_['mean_test_score']

array([0.85780355, 0.85799604, 0.85799604, ..., 0.86126601, 0.86165063,
       0.86357629])

In [34]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 5, 1, 5, 8, 2, 3, 1, 1, 0])

In [35]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 90,  91,  93, 117,  91,  95, 100, 124, 107,  92]))

In [36]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.27044196, 0.82177189, 0.15946517, 0.8303463 , 0.54125237,
       0.64031735, 0.52110015, 0.38671275, 0.14445648, 0.11836711])

In [39]:
params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),  # Uniform distribution between 0.0001 and 0.001
    'max_depth': randint(20, 50),                     # Random integer between 20 and 50
    'min_samples_split': randint(2, 25),              # Random integer between 2 and 25
    'min_samples_leaf': randint(1, 25),               # Random integer between 1 and 25
}

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1422d1670>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1422fd790>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1422d1190>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x142320eb0>},
                   random_state=42)

In [40]:
print(rs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}


In [41]:
print(np.max(rs.cv_results_['mean_test_score']))

0.8695428296438884


In [42]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.86
